In [19]:
# ------------------------------
# Install Transformers
# ------------------------------
!pip install transformers --quiet

# ------------------------------
# Imports
# ------------------------------
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from PIL import Image
import io
import torch
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display

# ------------------------------
# 1. CNN on Fashion MNIST
# ------------------------------
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train_cat, validation_split=0.2, epochs=3, batch_size=32, verbose=1)

# ------------------------------
# 2. Random Forest Tabular Demo
# ------------------------------
np.random.seed(42)
X_tab = np.array([
    [20,5,80],
    [50,3,50],
    [80,4,30],
    [35,2,70]
])
y_tab = np.array([0, 3, 5, 1])  # Fashion labels indices
rf_model = RandomForestClassifier(n_estimators=50, random_state=42)
rf_model.fit(X_tab, y_tab)

# ------------------------------
# 3. NLP - Transformers Sentiment
# ------------------------------
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# ------------------------------
# 4. Fashion Labels
# ------------------------------
fashion_labels = ["T-shirt/top","Trouser","Pullover","Dress","Coat",
                  "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

# ------------------------------
# 5. Widgets
# ------------------------------
upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
price_widget = widgets.IntText(value=50, description='Price:')
rating_widget = widgets.IntSlider(value=3, min=1, max=5, description='Rating:')
popularity_widget = widgets.IntSlider(value=50, min=1, max=100, description='Popularity:')
review_widget = widgets.Textarea(value='', description='Review:')
output_widget = widgets.Output()
predict_button = widgets.Button(description='Predict')

def on_predict_clicked(b):
    with output_widget:
        output_widget.clear_output()
        if len(upload_widget.value) == 0:
            print("Please upload an image.")
            return
        image_file = list(upload_widget.value.values())[0]['content']

        # CNN Prediction
        image = Image.open(io.BytesIO(image_file)).convert("L").resize((28,28))
        img = np.array(image)/255.0
        img = np.expand_dims(img, axis=(0,-1))
        cnn_pred = cnn_model.predict(img)
        cnn_class = fashion_labels[np.argmax(cnn_pred)]

        # ML Prediction
        tab_features = np.array([[price_widget.value, rating_widget.value, popularity_widget.value]])
        ml_class_idx = rf_model.predict(tab_features)[0]
        ml_class = fashion_labels[ml_class_idx]

        # NLP Prediction using Transformers
        nlp_result = sentiment_pipeline(review_widget.value)[0]
        nlp_label = nlp_result['label']  # 'POSITIVE' or 'NEGATIVE'

        print(f"CNN Prediction (Image): {cnn_class}")
        print(f"ML Prediction (Tabular): {ml_class}")
        print(f"NLP Sentiment: {nlp_label}")

predict_button.on_click(on_predict_clicked)

display(widgets.VBox([
    upload_widget,
    price_widget,
    rating_widget,
    popularity_widget,
    review_widget,
    predict_button,
    output_widget
]))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 123s 81ms/step - accuracy: 0.7652 - loss: 0.6490 - val_accuracy: 0.8687 - val_loss: 0.3556
Epoch 2/3
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 127s 71ms/step - accuracy: 0.8859 - loss: 0.3138 - val_accuracy: 0.8925 - val_loss: 0.2953
Epoch 3/3
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 50s 33ms/step - accuracy: 0.8980 - loss: 0.2761 - val_accuracy: 0.9042 - val_loss: 0.2717


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu
